In [1]:
L=!ls r_figures/
for l in L:
    print "![%s](r_figures/%s)"%(l,l)

![ClimateDataLogan.png](r_figures/ClimateDataLogan.png)
![ClimateDataOgden.png](r_figures/ClimateDataOgden.png)
![Map of Average snow depth.jpg](r_figures/Map of Average snow depth.jpg)
![PRCP_cdf.png](r_figures/PRCP_cdf.png)
![PRCP_cmp.png](r_figures/PRCP_cmp.png)
![P_norm_distribution.png](r_figures/P_norm_distribution.png)
![P_norm_mat.png](r_figures/P_norm_mat.png)
![SNOW.png](r_figures/SNOW.png)
![SNWD.png](r_figures/SNWD.png)
![SNWD_Neg_coeff1.png](r_figures/SNWD_Neg_coeff1.png)
![SNWD_Neg_coeff1_station.png](r_figures/SNWD_Neg_coeff1_station.png)
![SNWD_Neg_coeff2.png](r_figures/SNWD_Neg_coeff2.png)
![SNWD_Neg_coeff3.png](r_figures/SNWD_Neg_coeff3.png)
![SNWD_coeff1_CDF.png](r_figures/SNWD_coeff1_CDF.png)
![SNWD_coeff2_CDF.png](r_figures/SNWD_coeff2_CDF.png)
![SNWD_coeff3_CDF.png](r_figures/SNWD_coeff3_CDF.png)
![SNWD_coeff_1_CDF.png](r_figures/SNWD_coeff_1_CDF.png)
![SNWD_coeff_2_CDF.png](r_figures/SNWD_coeff_2_CDF.png)
![SNWD_coeff_3_CDF.png](r_figures/SNWD_coeff_3_CDF.png)
![

# Utah Weather Analysis

This is a report on the historical analysis of weather patterns in an area that approximately overlaps the area of the state of Utah. Specifically, we focus on the area around of Ogden and Logan where the stations in our dataset are mainly set.

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/). We partition the stations into 256 geographical rectangles, indexed from BBBBBBBB to SSSSSSSS. And each containing about 12,000 (station,year) pairs from which we choose the index of SBSSBSBB. And the locations of the stations on the map is as follows from which we could tell that most of the stations are located in the areas near the city of Ogden and Logan.

<p><img alt="Station_map.png" src="r_figures/Station_map.png"  /></p>


We focused on six measurements:
* **TMIN, TMAX:** the daily minimum and maximum temperature.
* **TOBS:** The average temperature for each day.
* **PRCP:** Daily Percipitation (in mm)
* **SNOW:** Daily snowfall (in mm)
* **SNWD:** The depth of accumulated snow.

## Sanity-check: comparison with outside sources

<p>We start by comparing some of the general statistics with graphs that we obtained from a site called <a href="http://www.usclimatedata.com/climate/ogden/utah/united-states/usut0385/2017/1" target="_blank">US Climate Data</a> The graph below shows the daily minimum and maximum temperatures for each month, as well as the total precipitation for each month in the city of Ogden and Logan.</p>

<p>&nbsp;</p>

<div style="float:left;"> 
<img alt="ClimateDataOgden.png" src="r_figures/ClimateDataOgden.png" style="height:300px; width:400px" />
</div>
<img alt="ClimateDataLogan.png" src="r_figures/ClimateDataLogan.png" style="height:300px; width:400px" />


<p>&nbsp;</p>

<p>We see that the min and max daily&nbsp;temperature agree with the ones we got from our data, once we translate Fahrenheit to Centigrade. And from the comparison of the figure from our dataset and the figure from US Climate Data, we could find they share quite a similar pattern. The hottest month is July where the average highest temperature could reach over $30^\circ$C and lowest could reach $13^\circ$C. And the coldest month is January where   the average highest temperature could reach over $0^\circ$C and lowest could reach -$10^\circ$C.</p>

<p>&nbsp;</p>

<p><img alt="TMIN&TMAX.png" src="r_figures/TMIN&TMAX.png" style="height:300px; width:800px" /></p>

<p>To compare the precipitation, according to our analysis the average rainfall is 1.5 mm/day which translates to about 45 mm/month. According to US-Climate-Data the average rainfall is closer to 48 inch per month. And both the figures show the pattern that the precipitation would dramatically drop in July and August when the precipitation is only about 0.5 mm/day(15 mm/month)</p>

<p>&nbsp;<img alt="PRCP_cmp.png" src="r_figures/PRCP_cmp.png" style="height:300px; width:450px" /></p>


## General PCA analysis

For each of the six measurement, we compute the percentage of the variance explained as a function of the number of eigen-vectors used.

### Percentage of variance explained.
![VarExplained11.png](r_figures/varExplained1.png)
We see that the top 5 eigen-vectors explain 37% of variance for TMIN, 55% for TOBS and 37% for TMAX.

We conclude that of the three, TOBS is best explained by the top 5 eigenvectors. This is especially true for the first eigen-vector which, by itself, explains 48% of the variance.

![VarExplained22.png](r_figures/varExplained2.png)

The top 5 eigenvectors explain 13% of the variance for PRCP and 11% for SNOW. Both are low values. On the other hand the top 5 eigenvectors explain 90% of the variance for SNWD especially for the first eigenvector which explains about 70% of the variance. This means that these top 5 eigenvectors capture most of the variation in the snow signals. Based on that we will dig deeper into the PCA analysis for snow-depth.

It makes sense that SNWD would be less noisy than SNOW. That is because SNWD is a decaying integral of SNOW and, as such, varies less between days and between the same date on different years. And therefore, in the next section, we decide to analyze the snow-depth with our statistical methods.

## Analysis of Snow Depth

We choose to analyze the eigen-decomposition for snow-depth because the first 3 eigen-vectors explain 80% of the variance.

### Snow Depth Map

First we show the distribution of daily snow depth of each station on the map as follows. WE concatenate the snow depth for each day in each year in increasing order so as to generate a vector of length 40515 days indicating the snow depth of each day. Therefore, our data for snow depth is a matrix of (46, 40515) where NaN exists representing the missing records. The radius of the circle in the map represents the standard deviation and the color represents the average of the daily snow depth. Then blue circles represent average daily snow depth less than 30 mm. The orange ones represent average daily snow depth in the interval of [30 mm,60 mm]. And the red circles represent average daily snow depth more than 60 mm.


<p>&nbsp;<img alt="SNWD_map.png" src="r_figures/SNWD_map.png" style="height:600px; width:830px" /></p>

From the figure above, we could tell that the average snow depth in the north is generally more than that in the south. And the red circles representing the large snow depth usually occurs inside the mountains or in the far north. Also we notice that the red circles usually have the larger radius which means larger variance of daily snow depth at these stations. Therefore, we could make the primitive conclusion that the large average daily snow depth could come from a few heavy snow seasons.

### PCA Analysis

Then, we graph the mean and the top 3 eigen-vectors.

We observe that the snow season is from mid-november to the start of march, where the middle of February marks the peak of the snow-depth.

<p>&nbsp;<img alt="SNWD_mean_eig.png" src="r_figures/SNWD_mean_eig.png" style="height:450px; width:800px" /></p>

Next we interpret the eigen-functions. The first eigen-function (eig1) has a shape very similar to the mean function while it is symmetric to the mean function about the x=0 axis. The interpretation of this shape is that eig1 represents the overall amount of snow below/above the mean, but without changing the distribution over time. And as the shape of the first eigen-function is similar to the mean function, that could explain why the first eigenvector could explain about 70% of the variance.

**eig2 and eig3** are similar in the following way. They all oscilate between positive and negative values. In other words, they correspond to changing the distribution of the snow depth over the winter months, but they don't change the total (much).

They can be interpreted as follows:
* **eig2:** less snow in jan - mid feb, more snow in mid feb-march and pretty much less snow in nov - dec.
* **eig3:** less snow in jan - mid feb, much more snow in march and slightly more snow in nov - dec.


### Examples of reconstructions

#### Coeff1
Coeff1: most positive
![SNWD_Pos_coeff1.png](r_figures/SNWD_pos_coeff1.png)
Coeff1: most negative
![SNWD_neg_coeff1.png](r_figures/SNWD_Neg_coeff1.png)
From the 4 figures with large positive values of coeff1, we could find that the target line is lower than the mean line which indicates that the large positive value of coeff1 reveals the pattern that the snow in this station this year would be lower than the average. And therefore vice versa, low values of coeff1 would correspond to much more snow than the average.

#### Coeff2
Coeff2: most positive
![SNWD_Pos_coeff2.png](r_figures/SNWD_pos_coeff2.png)
Coeff2: most negative
![SNWD_neg_coeff2.png](r_figures/SNWD_Neg_coeff2.png)

And from the 4 figures of large positive values of coeff2, we could find the peak of target snow fall was generally later than the peak of the mean line during spring which indicates that the large positive value of coeff1 reveals the pattern of a late snow season during spring(most of the snowfall is after mid feb). From the 4 figures of large negative values for coeff2, we find the snow fall of the target line during the late winter is much higher than the average which shows the pattern that much more snow during late winter(nov - dec).
#### Coeff3
Coeff3: most positive
![SNWD_Pos_coeff3.png](r_figures/SNWD_pos_coeff3.png)
Coeff3: most negative
![SNWD_neg_coeff3.png](r_figures/SNWD_Neg_coeff3.png)

Similar to coeff2, from the 4 figures of large positive values of coeff3, we could find the peak of target snow fall was generally later than the peak of the mean line during spring which indicates that the large positive value of coeff1 reveals the pattern of a late snow season during spring(most of the snowfall is after mid feb). And we also observe that the snow fall of the target line during the late winter is much higher than the average which shows the pattern that much more snow during late winter(nov - dec). And negative values for coeff3 correspond to an early snow season during spring (most of the snow is before mid-feb).



### Analysis of Distribution of Coefficients and Residuals

As we've found out through analysis the general pattern shown by each coefficient in the last section, we continue to look into the distribution of the coefficient in each station each year to get an overview of the snow depth pattern.


#### Coeff1

<div style="float:left;"> 
<img alt="SNWD_coeff1_CDF.png" src="r_figures/SNWD_coeff1_CDF.png" style="height:300px; width:400px"/>
</div>
<img alt="SNWD_res1_CDF.png" src="r_figures/SNWD_res1_CDF.png" style="height:300px; width:450px"/>

According to the distribution of res_1, we find that the distribution is quite uniform from 0 to 1 and a few outliers that exceed 1. So we are not able to extract some significant information from the distribution of res_1.

According to the distribution of coeff1, we know that about 60% of the records having more snow than the average while about 40% of the records having less snow than the average. Also from the bottom left of the distribution we could find that in a few extreme records, the coeff1 are very low which indicates the abnormal large quantity of snow in a specific station during a specific year. And from the top right of the distribution we could find that in a few extreme records, the coeff1 are quite high which indicates the abnormal small quantity of snow in a specific station during a specific year. The station and year with abnormal quantity of snow is in the following figures. And we could tell from the figure that large amount of snow usually happens in recent 20 year while the small amount of snow usually happens long times ago.

##### Records with small SNWD
![SNWD_Pos_coeff1_station.png](r_figures/SNWD_pos_coeff1_station.png)
##### Records with large SNWD
![SNWD_neg_coeff1_station.png](r_figures/SNWD_Neg_coeff1_station.png)

#### Coeff2

<div style="float:left;"> 
<img alt="SNWD_coeff2_CDF.png" src="r_figures/SNWD_coeff2_CDF.png" style="height:300px; width:400px"/>
</div>
<img alt="SNWD_res2_CDF.png" src="r_figures/SNWD_res2_CDF.png" style="height:300px; width:400px"/>

According to the distribution of res_2, we find that again the distribution is quite uniform from 0 to 1 and a few outliers that exceed 1 a little bit.

According to the distribution of coeff2, we know that about half of the records having late snow during spring while the other half of the records having early snow during spring and much more snow during late winter. Also from the bottom left of the distribution we could find that in a few extreme records, the coeff2 are very low which indicates the obvious pattern of late snow during spring in a specific station during a specific year. And from the top right of the distribution we could find that in a few extreme records, the coeff2 are quite high which indicates the large quantity of snow during late winter in a specific station during a specific year.

#### Coeff3

<div style="float:left;"> 
<img alt="SNWD_coeff3_CDF.png" src="r_figures/SNWD_coeff3_CDF.png" style="height:300px; width:400px"/>
</div>
<img alt="SNWD_res3_CDF.png" src="r_figures/SNWD_res3_CDF.png" style="height:300px; width:400px"/>

According to the distribution of res_3, we find that again the distribution is quite uniform from 0 to 1 and a few outliers that exceed 1 a little bit.

According to the distribution of coeff3, we find that the distribution are quite symmetric about the x=0 axis and most of them(about 90%) are in the interval of [-1000,1000]. And again from the bottom left of the distribution we could find that in a few extreme records, the coeff3 are very low which indicates the obvious pattern of late snow during spring and large quantity of snow during late winter in a specific station during a specific year. And from the top right of the distribution we could find that in a few extreme records, the coeff2 are quite high which indicates an early snow season during spring in a specific station during a specific year.





### Analyze whether SNWD varies more from year to year or from place to place.

In the previous section we see the variation of Coeff1, which corresponds to the total amount of snow, with respect to location. We now estimate the relative importance of location-to-location variation relative to year-by-year variation.
These are measured using the fraction by which the variance is reduced when we subtract from each station/year entry the average-per-year or the average-per-station respectively. Here are the results:

#### coeff_1 
total RMS = 1542.4295994 

RMS removing mean-by-station = 1167.01854244, fraction explained= 0.243389427372 

RMS removing mean-by-year = 1181.70634773, fraction explained= 0.233866914775

#### coeff_2 
total RMS = 601.90862572

RMS removing mean-by-station = 575.299716846, fraction explained= 0.0442075553272

RMS removing mean-by-year = 346.048423438, fraction explained= 0.425081468099

#### coeff_3 
total RMS = 598.320831818

RMS removing mean-by-station = 545.039058008, fraction explained= 0.0890521789937

RMS removing mean-by-year = 393.790733512, fraction explained= 0.341840175755

We see that the variation by year explains about the same as the variation by station for coeff_1, which has to do with the total snowfall, vs. coeff_2,3 which, as we saw above have to do with the timing of snowfall where the variation by year explains more than the variation by station. We see that for coeff_2,3 the stations explain 5-10% of the variance while the years explain 30-40%.

### Conclusion

In this section, we analyze the snow depth in the area of Ogden and Logan with statistical tools. First with eigen-decomposition, we find that the first 3 eigenvectors could explains most of the variance in the snow depth data. Then we reconstruct the original data with the first 3 eigenvectors and find the real-world meaning of each coefficient through the pattern shown in the figures. Finally we find that the timing of snowfall mainly varies from year to year rather than from station to station.


## Analysis of correlation between precipitation across locations

In the last section, we focus on the snow depth during a whole year in different stations and years. And in this section, we try to dig into the correlation between precipitation across different stations. It is likely to be hard to find correlations between the amount of rain on the same day in different stations. Because amounts of rain vary a lot between even close locations. It is more reasonable to try to compare whether or not it rained on the same day in different stations. As we see from the graph below, among 828638 days in our region with records, it rains in about 78% of the days.

<p>&nbsp;<img alt="PRCP_cdf.png" src="r_figures/PRCP_cdf.png" style="height:300px; width:450px" /></p>

### Definition of statistical test

For each station, we concatenate the precipitation for each day in each year in increasing order so as to generate a vector of length 43435 days indicating which days rain. Therefore, our data for precipitation is a matrix of (93,43435) where NaN exists representing the missing records.

Then fix two stations. We restrict our attention to the days for which we have measurements for both stations, and define the following notation:
* $m$ : the total number of days (for which we have measurements for both stations).
* $n_1$ : the number of days that it rained on station 1
* $n_2$ : the number of days that it rained on station 2
* $l$ : the number of days that it rained on both stations.

We want to calculate the probability that the number of overlap days is $l$ given $m,n_1,n_2$.

The answer is:
$$
P = {m \choose l,n_1-l,n_2-l,m-n_1-n_2+l} /{m \choose n_1}{m \choose n_2}
$$

Where
$$
{m \choose l,n_1-l,n_2-l,m-n_1-n_2+l} = \frac{m!}{l! (n_1-l)! (n_2-l)! (m-n_1-n_2+l)!}
$$

We use the fact that $\Gamma(n+1) = n!$ and denote $G(n) \doteq \log \Gamma(n+1)$
$$
\log P = \left[G(m) - G(l) -G(n_1-l) -G(n_2-l) -G(m-n_1-n_2+l) \right] - 
\left[G(m)-G(n_1)-G(m-n_1)\right] - \left[G(m)-G(n_2)-G(m-n_2)\right]
$$
Which slightly simplifies to 
$$
\log P = -G(l) -G(n_1-l) -G(n_2-l) -G(m-n_1-n_2+l) - G(m)+G(n_1)+G(m-n_1) +G(n_2)+G(m-n_2)
$$

The log probability scales with $m$ the length of the overlap. So to get a per-day significance we consider $
\frac{1}{m} \log P $

### correlations matrix

We compute the P_value for each pair of station and get the distribution as in the following figures.


<p>&nbsp;<img alt="P_norm_distribution.png" src="r_figures/P_norm_distribution.png" style="height:300px; width:450px" /></p>

We notice that there is a single peak at 0.4 with count of 93, which is the P_norm of each station with itself. And for the rest of the distribution, we could tell that most of the pairs have the P_norm between 0.05 and 0.15 while a few have the P_norm above 0.3 indicating strong correlation which are the pairs we should focus on. And to clearly see the connection between the stations, we have the following figure.

<p>&nbsp;<img alt="P_norm_mat.png" src="r_figures/P_norm_mat.png" style="height:400px; width:400px" /></p>

And from the top left corner of this figure, we could find 3 strongly correlated group of stations in the first 18 stations, that is station 1 to 7, 9 to 14 and 15 to 17. And then we find the corresponding locations of these stations on the map as shown in the following figure.



<p>&nbsp;<img alt="station_map_group.png" src="r_figures/Station_map_group.png" style="height:500px; width:600px" /></p>


First in the map of the area near Ogden, we could find the 5 orange circles inside the mountain areas which shows correlated precipitation patterns. And there are 3 red circles outside the mountain areas which between the mountain and the Great Salt Lake. And in the map of the area near Logan, we could find the two red circles near the center of Logan city and two green circles in the far north of Logan city.

Therefore, we could draw the primitive conclusion that the geographical location of the station plays a significant role to the precipitation pattern of the station.

### PCA analysis

Then to illustrate the specific features of each cluster of the stations, we choose the PCA analysis to represent the features of each station with the coefficient after the eigen-decomposition. Again, we first figure out the percentage of the variance explained by each eigenvector and find out that the first 4 eigenvectors could explain about 70% of the variance. So in the following analysis, we just use the first 4 eigenvectors.

<p>&nbsp;<img alt="varExplained3.png" src="r_figures/varExplained3.png" style="height:300px; width:450px" /></p>

### Geographical distribution of top 4 coefficients.

Then we draw the locations of the stations on the map again. But instead of simple circle, we use polygons consisted of 4 triangles to represent the 4 coefficients for first 4 eigenvectors at each station. For positive coefficients, we use the hollow triangles without filling color and for negative ones we use the triangle with filling color. The size of the triangles represents the absolute value of the coefficient. And the 4 coefficient is colored in blue, orange, green and red respectively. The map is as follows.

<p>&nbsp;<img alt="coeff_map.png" src="r_figures/coeff_map.png" style="height:500px; width:800px" /></p>

And we could find some geographical pattern for the stations with similar coefficient distribution. 

* For instance, we find that at the bottom left of the map, there are several large green hollow triangles near the bank of Great Salt Lake. 
* Then between the Great Salt Lake and the mountain area, for those areas near the center of Ogden city, there are several large orange color-filled triangles with hollow red ones. 
* And inside the mountain area near Ogden city, there are plenty of large blue color-filled triangles.(We found these stations highly correlated in the previous section!!!)
* As for the area near the center of Logan, there are several mixed pattern so that no clear common features could be extracted. 
* But when it comes to the areas far north to Logan, there are again quite a few large blue color-filled triangles.

### Conclusion

From the analysis above, we could find that the precipitation pattern varies with different geographical features. The stations near each other in geographical locations have higher similarity respect to the precipitation pattern.